<center>
<table>
  <tr>
    <td><img src="https://portal.nccs.nasa.gov/datashare/astg/training/python/logos/nasa-logo.svg" width="100"/> </td>
     <td><img src="https://portal.nccs.nasa.gov/datashare/astg/training/python/logos/ASTG_logo.png?raw=true" width="80"/> </td>
     <td> <img src="https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png" width="130"/> </td>
    </tr>
</table>
</center>

        
<center>
<h1><font color= "blue" size="+3">ASTG Python Courses</font></h1>
</center>


---

<center>
    <h1><font color="red">Image Classification with Scikit-Learn</font></h1>
</center>

## Useful Links

- <a href="https://www.dataquest.io/blog/sci-kit-learn-tutorial/">Scikit-learn Tutorial: Machine Learning in Python</a>
- <a href="https://debuggercafe.com/image-classification-with-mnist-dataset/">Image Classification with MNIST Dataset</a>
- <a href="https://davidburn.github.io/notebooks/mnist-numbers/MNIST%20Handwrititten%20numbers/">MNIST handwritten number identification</a>

In [ ]:
try:
    import google.colab
    print("Running in Google Colab")
except:
    print("Not running in Google Colab")
else:
    print("Installing modules in Google Colab")
    !pip install seaborn
    !pip install -U scikit-learn

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import sklearn
from sklearn.base import clone

In [ ]:
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
print(f"Numpy version:        {np.__version__}")
print(f"Pandas version:       {pd.__version__}")
print(f"Seaborn version:      {sns.__version__}")
print(f"Scikit-Learn version: {sklearn.__version__}")

# <font color="red"> MNIST Dataset</font>

- The <A HREF="https://en.wikipedia.org/wiki/MNIST_database"> MNIST database</A> (Modified National Institute of Standards and Technology database) is a large database of handwritten digits that is commonly used for training various image processing systems.
- The database is also widely used for training and testing in the field of machine learning.
- The dataset we will be using contains 70000 images of handwritten digits among which 10000 are reserved for testing.
- This dataset is  suitable for anyone who wants to get started with image classification using Scikit-Learn. 

## <font color="blue"> Obtain the Dataset</font>

In [ ]:
from sklearn.datasets import fetch_openml
mnist_data = fetch_openml('mnist_784', version=1)

In [ ]:
print(mnist_data.DESCR)

## <font color="blue"> Features of the Dataset</font>

In [ ]:
print("Keys: ", mnist_data.keys())

**Note that the `data` and `target` already separated.**

In [ ]:
print(f"Shape of Data: {mnist_data.data.shape}")

In [ ]:
print(f"Datatype of Data: {type(mnist_data.data)}")

In [ ]:
print(f"Shape of the Target Data: {mnist_data.target.shape}")

In [ ]:
print(f"Datatype of Target Data: {type(mnist_data.target)}")

In [ ]:
print(f"Feature Names: {mnist_data.feature_names}")

In [ ]:
print(f"Url: {mnist_data.url}")

Extract the feature and target arrays:

In [ ]:
np_data, np_target = mnist_data['data'], mnist_data['target']

In [ ]:
print(f' Shape of data:   {np_data.shape}')
print(f' Shape of target: {np_target.shape}')

**Checking the Data**

In [ ]:
len(np.unique(np_data))

In [ ]:
np_data.values[0]

In [ ]:
len(np.unique(np_data.values[0]))

**Checking the Target**

In [ ]:
print(f"Datatype of the target values: {np_target.dtype}")

In [ ]:
np_target[0]

In [ ]:
type(np_target[0])

Print a few values:

In [ ]:
print(np_target[0:5])

Changing the labels from string to integers:

In [ ]:
np_target = np_target.astype(np.uint8)

In [ ]:
print(np_target[0:5])

Print the number of unique labels:

In [ ]:
np.unique(np_target)

In [ ]:
np_target.value_counts()

In [ ]:
np_target.value_counts().sum()

In [ ]:
total = np_target.value_counts().sort_values(ascending=False)
percent = (np_target.value_counts()/np_target.count()).sort_values(ascending=False)*100
percent_data = pd.concat([total, percent], axis=1, 
                         keys=['Total', 'Percent'])
percent_data

<font color="blue"> 
There are 70000 numbers, each stored as an array of 784 numbers depicting the opacity of each pixel, it can be displayed by reshaping the data into a 28x28 array and plotting using matplotlib. 
</font>

In [ ]:
some_index = 15657
some_digit = np_data.values[some_index]
some_digit_image = some_digit.reshape(28,28)

plt.imshow(some_digit_image, 
           cmap = matplotlib.cm.binary, 
           interpolation='nearest')
plt.axis=('off')

Let us find the target for row `some_index`:

In [ ]:
np_target[some_index]

In [ ]:
np_data.values.shape

#### Display a few images

In [ ]:
import random

def display_digits(X, y):
    """
      Given an array of images of digits X and 
      the corresponding values of the digit y,
      this function plots 96 unique randomly selected images 
      and their values.
    """
    # Figure size (width, height) in inches
    fig = plt.figure(figsize=(8, 6))

    # Adjust the subplots 
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, 
                        hspace=0.05, wspace=0.05)

    num_images = X.shape[0]
    
    num_selected_images = 96
    row_indices = random.sample(range(num_images), num_selected_images)
    
    i = 0
    for idx in row_indices:
        # Initialize the subplots: 
        # Add a subplot in the grid of 8 by 12, at the i+1-th position
        ax = fig.add_subplot(8, 12, i + 1, xticks=[], yticks=[])
        
        # Display an image at the i-th position
        ax.imshow(X[idx].reshape(28, 28), cmap=plt.cm.binary, 
                  interpolation='nearest')
       
        # label the image with the target value
        ax.text(0, 7, str(y[idx]))
        i += 1

    # Show the plot
    plt.show()

In [ ]:
display_digits(np_data.values, np_target)

# <font color="red">Model Selection Process</font>

![fig_skl](https://miro.medium.com/max/1400/1*LixatBxkewppAhv1Mm5H2w.jpeg)
Image Source: Christophe Bourguignat

- A Machine Learning algorithm needs to be trained on a set of data to learn the relationships between different features and how these features affect the target variable. 
- We need to divide the entire data set into two sets:
    + Training set on which we are going to train our algorithm to build a model. 
    + Testing set on which we will test our model to see how accurate its predictions are.
    
Before we create the two sets, we need to identify the algorithm we will use for our model.
We can use the `machine_learning_map` map (shown at the top of this page) as a cheat sheet to shortlist the algorithms that we can try out to build our prediction model. 

# <font color="red">Separating the Training and Testing Set</font>

- The first 60000 (among the 70000) images are used for training.
- The remaining 10000 images are used for validations

In [ ]:
num_train = 60000

X_train = np_data.values[:num_train]
X_test  = np_data.values[num_train:]
y_train = np_target[:num_train]
y_test  = np_target[num_train:]

In [ ]:
print(f' Train Data:  {X_train.shape}')
print(f' Test Data:   {X_test.shape}')
print(f' Train label: {y_train.shape}')
print(f' Test Label:  {y_test.shape}')

**Shuffle the training set:**

In [ ]:
nn = X_train.shape[0]
shuffle_index = np.random.permutation(nn)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

# <font color="red"> ML models

##  <font color="blue">Binary classifier</font>

- Binary classification means there are two classes to work with that relate to one another as `true` and `false`.
- Here, we want to identify a single digit: looking at `9`s.
- The classification will tell us if we have a `9` (true) or not (false).

**Set the target arrays as boolean arrays:** true if 9 otherwise false.

In [ ]:
y_train_9 = (y_train == 9)
y_test_9 = (y_test == 9)

#### Create and train the model

- We use the SGDClassifier that applies regularized linear model with SGD (Stochastic Gradient Descent) learning to build an estimator.
- It works by progressively changing model parameters in the direction of a loss function's sharpest gradient. 
- The method helps building an estimator for classification and regression problems.

```python
SGDClassifier(loss='hinge', *, penalty='l2', 
              alpha=0.0001, l1_ratio=0.15, fit_intercept=True, 
              max_iter=1000, tol=0.001, shuffle=True, 
              verbose=0, epsilon=0.1, n_jobs=None, random_state=None, 
              learning_rate='optimal', eta0=0.0, power_t=0.5, 
              early_stopping=False, validation_fraction=0.1, 
              n_iter_no_change=5, class_weight=None, warm_start=False, 
              average=False)
```

In [ ]:
sgd_clf =SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)

In [ ]:
sgd_clf.fit(X_train, y_train_9)

**Make an initial prediction:**

In [ ]:
some_index = 15657
some_digit = np_data.values[some_index]
print(np_target[some_index])

In [ ]:
some_digit_predict = sgd_clf.predict([some_digit])

In [ ]:
some_digit_predict

**Measuring accuracy using cross validation**

- The `stratifiedKfold` class performs stratified sampling to produce folds that contain a representative ratio of each class. 
- At each iteration the code creates a clone of the classifier, trains that clone on the training fold and then makes predictions on the test fold. 
- It then counts the number of correct predictions and outputs the ratio of correct predictions.

In [ ]:
n_splits = 3
skfolds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

kfold_scores = list()
idx = 0
for train_index, test_index in skfolds.split(X_train, y_train_9):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = y_train_9[train_index]
    X_test_fold = X_train[test_index]
    y_test_fold = y_train_9[test_index]
    
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    scr = n_correct/len(y_pred)
    kfold_scores.append(scr)
    print(f"Test: {idx} -- Score: {scr}")
    idx += 1

In [ ]:
print(f"{n_splits}-fold average score: {np.mean(np.array(kfold_scores))}")

In [ ]:
validation = cross_val_score(sgd_clf, X_train, y_train_9, 
                             cv=3, 
                             scoring='accuracy', 
                             verbose=1)

print(validation)

The sklearn cross_val_score in action returning the same result.

In [ ]:
accuracy = (sum(np_target==9)/len(np_target))*100
print(f'{np.max(validation)*100}% accuracy might not as impressive as it sounds \n where there are {accuracy :.2f}% of 9s in the dataset')

**Confusion matrix**

- A confusion matrix is a tabular summary of the number of correct and incorrect predictions made by a classifier. 
- It can be used to evaluate the performance of a classification model through the calculation of performance metrics like accuracy, precision, recall, and F1-score.
- The confusion matrix is a much better way to evaluate the performance of a classifier, especially when there is a skewed dataset as we have here with only 10% of the dataset being the target.
- Each row represents a class, each column a prediction:
   * The first row is negative cases (non-9s) with the top left containing all the correctly classified non-9s (True Negatives), the top right the 9s incorrectly classified as non-9s (False-Positves).
   * The second row represents the positive class, 9s in this case, bottom left contains the 9s incorrectly classified as non-9s (False Negatives), the bottom right containing the correctly classified 9s (True Positives)


|     | Actual |      |
| --- |:---   |:--- |
| **Prediction** | True Positive  | False Positive |
|                | False Negative | True Negative |

We first need a set of predictions to compare to the actual targets:

In [ ]:
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_9, cv=3)

In [ ]:
cf_matrix = metrics.confusion_matrix(y_train_9, y_train_pred)

print(f"Confusion Matrix: \n{cf_matrix}")
print(f"\n Number of images: {np.sum(cf_matrix)}")

We can visualize the confusion matrix:

In [ ]:
cm = metrics.confusion_matrix(y_train_9, y_train_pred, 
                              labels=sgd_clf.classes_)
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, 
                                      display_labels=sgd_clf.classes_)
disp.plot() 

#### Precision/Recall
- Precision-Recall is a useful measure of success of prediction when the classes are very imbalanced.
- Precision measures the number of true positives (correctly classified 9s) as a ratio of the total samples classified as a 9: $\frac{T_P}{T_P + F_P}$
- Recall measures the number of true positives as a ratio of the total number of positives: $\frac{T_P}{T_P + F_N}$.
- The precision-recall curve shows the tradeoff between precision and recall for different threshold. 
  - A system with high recall but low precision returns many results, but most of its predicted labels are incorrect when compared to the training labels. 
  - A system with high precision but low recall is just the opposite, returning very few results, but most of its predicted labels are correct when compared to the training labels. 
  - An ideal system with high precision and high recall will return many results, with all results labeled correctly.

In [ ]:
y_scores = cross_val_predict(sgd_clf, X_train, y_train_9, cv=3, method='decision_function')

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(y_train_9, y_scores)

def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.figure(figsize=(12,8))
    plt.title('Precision and recall vs decision threshold')
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    plt.xlabel("Threshold")
    plt.legend(loc="upper left")
    plt.ylim([0,1])

plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

## <font color="blue">Models with the entire dataset</font>

- We will use the Stochastic Gradient Descent classifier (SGD). 
- Scikit-Learn’s SGDClassifier is a good starting point for linear classifiers. 
- Using the loss parameter we will see how Support Vector Machine (Linear SVM) and Logistic Regression perform for the same dataset.


### <font color="green"> Linear Support Vector Machine (SVM)</font>
- SVMs are a set of supervised learning methods used for classification, regression and outliers detection.
- We use linear SVM with stochastic gradient descent (SGD) learning.
    - The gradient of the loss is estimated each sample at a time and the model is updated along the way with a decreasing strength schedule.
    - To use the Linear SVM Classifier, we need to set the loss parameter to `hinge`.

```python
LinearSVC(penalty='l2', loss='squared_hinge', *, dual='auto', 
          tol=0.0001, C=1.0, multi_class='ovr', 
          fit_intercept=True, intercept_scaling=1, 
          class_weight=None, verbose=0, random_state=None, 
          max_iter=1000)
```

In [ ]:
#sgd_clf = SGDClassifier(loss='hinge', random_state=42)
svm_clf = LinearSVC(loss='hinge', random_state=42)
svm_clf.fit(X_train, y_train)

- Before testing the model, it is a good practice to first see the cross-validation scores on the training data. 
- That you will give you a very good projection of how the model performs.

In [ ]:
valid = cross_val_score(svm_clf, X_train, y_train, cv=3, scoring='accuracy')

In [ ]:
min_val = np.min(valid)*100
max_val = np.max(valid)*100
print(f'For three-fold Cross-Validation you are getting around: {min_val}%-{max_val}%')

We can now compute the actual test scores:

In [ ]:
scoreSVM = svm_clf.score(X_test, y_test)
print("Test score of the Linear SVM: ", scoreSVM)

In [ ]:
y_predict = svm_clf.predict(X_test)

cm = metrics.confusion_matrix(y_test, y_predict, 
                              labels=svm_clf.classes_)
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, 
                                      display_labels=svm_clf.classes_)
disp.plot();

### <font color="green"> Logistic Regression</font>

```python
LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, 
                   C=1.0, fit_intercept=True, intercept_scaling=1, 
                   class_weight=None, random_state=None, 
                   solver='lbfgs', max_iter=100, multi_class='deprecated', 
                   verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
```

In [ ]:
#sgd_clf = SGDClassifier(loss='log_loss', random_state=42)
lr_clf = LogisticRegression(random_state=42)
lr_clf.fit(X_train, y_train)

In [ ]:
cross_val_score(lr_clf, X_train, y_train, cv=3, scoring='accuracy')

In [ ]:
score_lr = lr_clf.score(X_test, y_test)
print(f"Test score of the Logistic Regression: {score_lr}")

In [ ]:
y_predict = lr_clf.predict(X_test)

cm = metrics.confusion_matrix(y_test, y_predict, 
                              labels=lr_clf.classes_)
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, 
                                      display_labels=lr_clf.classes_)
disp.plot();

### <font color="green">Random Forest Classifier</font>

- Random forests is a supervised learning algorithm. 
- A forest is comprised of trees. 
- It is said that the more trees it has, the more robust a forest is. 
- Random forests creates decision trees on randomly selected data samples, gets prediction from each tree and selects the best solution by means of voting. 
- It also provides a pretty good indicator of the feature importance.

In [ ]:
n_estimators = 500
forest = RandomForestClassifier(n_estimators = n_estimators)

In [ ]:
forest = forest.fit(X_train, y_train)

In [ ]:
forest_output = forest.predict(X_test)

Calculate accuracy on the prediction:

In [ ]:
print(f"Random Forest with n_estimators = {n_estimators}")
print(accuracy_score(y_test, forest_output))

Display few true images against predictions:

In [ ]:
display_digits(X_test, forest_output)

In [ ]:
cm = metrics.confusion_matrix(y_test, forest_output, 
                              labels=forest.classes_)
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, 
                                      display_labels=forest.classes_)
disp.plot()

### <font color="green">Gradient Boosting Classifier</font>

In [ ]:
clf = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, 
                                 max_depth=1, random_state=0).fit(X_train,y_train)

In [ ]:
gbc_output = clf.predict(X_test) 

Calculate accuracy on the prediction:

In [ ]:
print(f"Gradient Boosting Accuracy: {accuracy_score(y_test, gbc_output)}")

Display few true images against predictions:

In [ ]:
display_digits(X_test, gbc_output)

In [ ]:
cm = metrics.confusion_matrix(y_test, gbc_output, 
                              labels=clf.classes_)
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, 
                                      display_labels=clf.classes_)
disp.plot()

### <font color="green">MLP Classifier: Neural Network</font>

- The Multi-layer Perceptron (MLP) classifier relies on an underlying Neural Network to perform the task of classification.
- MLP trains on two arrays:
   - array `X` of size (`n_samples`, `n_features`), which holds the training samples represented as floating point feature vectors; and
   - array `y` of size (`n_samples`,), which holds the target values (class labels) for the training samples.

```python
MLPClassifier(hidden_layer_sizes=(100,), activation='relu',  
              solver='adam', alpha=0.0001, batch_size='auto', 
              learning_rate='constant', learning_rate_init=0.001, 
              power_t=0.5, max_iter=200, shuffle=True, random_state=None, 
              tol=0.0001, verbose=False, warm_start=False, momentum=0.9, 
              nesterovs_momentum=True, early_stopping=False, 
              validation_fraction=0.1, beta_1=0.9, beta_2=0.999, 
              epsilon=1e-08, n_iter_no_change=10, max_fun=15000)
```

Here `hidden_layer_sizes=(100,)` means one input layer, one hidden layer with 100 nodes, one output layer.

We choose a range of values for:

- `alpha`: Strength of the L2 regularization term. The L2 regularization term is divided by the sample size when added to the loss.
- `max_iter`: Maximum number of iterations.

to run the model. We access how these parameters affect the model accuracy.

In [ ]:
from time import time
i = 0
df = pd.DataFrame(columns = ['alpha','max_iter','train_acc','test_acc','train_time'])
for a in [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]:
    for mi in [10, 100, 200, 500, 1000, 2000]:
        st = time()
        mlp = MLPClassifier(alpha=a, max_iter=mi)
        mlp.fit(X_train, y_train)
        end = time() - st
        
        acc_tr = accuracy_score(y_train, mlp.predict(X_train)) # Train Accuracy
        acc = accuracy_score(y_test, mlp.predict(X_test)) # Test Accuracy
        df.loc[i] = [a, mi, acc_tr, acc, end]
        print(f"| {i:2d} | {a:7.5f} | {mi:4d} | {acc_tr:5.4f} | {acc:5.4f} | {end:7.4f} |")
        i += 1

In [ ]:
df

#### Neural networks with two hidden layers

We vary the number of nodes in hidden layers and determine the train and test accuracies.

In [ ]:
layer_sizes = [25, 50, 75, 100, 150, 200, 250, 300]
N = len(layer_sizes)

In [ ]:
acc_test = list()
acc_train = list()
timelog = list()
for l in layer_sizes:
    beg_time = time()
    mlp = MLPClassifier(alpha=0.0001, max_iter=10, 
                        hidden_layer_sizes=(l,l),
                        activation='relu')
    mlp.fit(X_train, y_train)
    elapsed_time = time() - beg_time
        
    a_train = accuracy_score(y_train, mlp.predict(X_train))
    a_test = accuracy_score(y_test, mlp.predict(X_test))

    acc_train.append(a_train)
    acc_test.append(a_test)
    timelog.append(elapsed_time)

In [ ]:
l2 = np.arange(N)
acc_test = np.array(acc_test)
acc_train = np.array(acc_train)
acc_diff = acc_train - acc_test

fig, ax1 = plt.subplots(figsize=(10, 5))
ax1.plot(l2, acc_test, color="red", label="Testing Accuracy")
ax1.plot(l2, acc_train, color="green", label="Training Accuracy")
ax1.set_xticks(l2, layer_sizes)
ax1.set_xlabel("Hidden Layer Nodes")
ax1.set_ylabel("Accuracy")
ax1.grid()

ax2 = ax1.twinx()
ax2.plot(l2, acc_diff, 'b:', label='Diff')
ax2.set_ylabel('Difference of the two accuracies')
#ax2.grid()

#plt.grid(axis='both')
fig.legend(loc="lower right")
plt.title('Accuracy versus Nodes in the Hidden Layer for MLPClassifier', fontsize=12)
fig.tight_layout()